## How To Perform a T1 Experiment

### Prerequisites
This guide assumes you have a configured `DeviceSetup` as well as `Qubit` objects with assigned parameters. Please see these guides (add links) if you need to create your setup and qubits for the first time. However, you can also run this notebook "as is" using an emulated session. If you are just getting started with the LabOne Q Applications Library, please don't hesitate to reach out to us at info@zhinst.com.

### Background

In this how-to guide, you'll perform a measurement to determine the relaxation time, $T_1$, for a given qubit using the `T1` experiment included in the LabOne Q Applications Library. In a $T_1$ experiment, you'll vary the delay time, $\tau$, between an initial excitation pulse and the subsequent measurement of the qubit's state. This allows you to observe how the qubit relaxes from the excited state, $\ket{1}$, back to the ground state, $\ket{0}$, over time.

For a qubit transitioning from the excited state, $\ket{1}$, to the ground state, $\ket{0}$, the population decay follows an exponential behavior according to:

$$\rho(\ket{1}) = e^{-\tau/T_1}$$



### Imports

You'll start by importing the T1 experiment from `laboneq_applications`, as well as `laboneq.simple` and a demo QPU and device setup to run in emulation mode.

In [ ]:
from laboneq.contrib.example_helpers.plotting.plot_helpers import plot_simulation
from laboneq.simple import *

from laboneq_applications.experiments import T1
from laboneq_applications.qpu_types.tunable_transmon.demo_qpus import demo_platform

### QPU and Device Setup

You'll generate six qubits with pre-defined parameters, as well as a `Device_Setup` consisting of a SHFQC+, HDAWG, and PQSC. If you already have your own `DeviceSetup` and qubits configured, you'll instead initialize the session using your setup.

In [ ]:
my_platform = demo_platform(6)

Then, you'll connect to the `Session`. Here we connect to an emulated one:

In [ ]:
session = Session(my_platform.setup)

session.connect(do_emulation=True)

### Running the T1 Workflow

You'll now make the experiment workflow and run:

In [ ]:
# our qubits live here in the demo setup:
qubits = my_platform.qpu.qubits

my_workflow = T1.experiment_workflow(
    session=session,
    qpu=my_platform.qpu,
    qubits=[qubits[0], qubits[1]],
    delays=[[0, 100e-9, 1000e-9], [0, 100e-9, 1000e-9]],
)

my_results = my_workflow.run()

#### Output Simulation

You can also inspect the compiled experiment and plot the simulated output:

In [ ]:
compiled_T1 = my_results.tasks["compile_experiment"].output  # noqa: N816
plot_simulation(
    compiled_T1,
    signal_names_to_show=["drive", "measure"],
    start_time=0,
    length=10e-6,
)

#### Inspecting the Source Code

You can inspect the source code of the `create_experiment` task defined in `T1` to see how the experiment pulse sequence is created:

In [ ]:
T1.create_experiment.src

### Changing the Options

We can give our T1 experiment options. First, inspect what they currently are:

In [ ]:
my_new_opts = T1.options()
my_new_opts

Then provide new options. This time, you'll remove the calibration traces and change the counts.

In [ ]:
my_new_opts.create_experiment.count = 2048
my_new_opts.create_experiment.use_cal_traces = False
my_new_opts

#### Run the workflow with updated options

Now, run the workflow with new options and inspect the simulated output. You'll notice that the calibration traces are no longer there.

In [ ]:
my_new_workflow = T1.experiment_workflow(
    session=session,
    qpu=my_platform.qpu,
    qubits=[qubits[0], qubits[1]],
    delays=[[0, 100e-9, 1000e-9], [0, 100e-9, 1000e-9]],
    options=my_new_opts,
)

my_new_results = my_new_workflow.run()
new_compiled_T1 = my_new_results.tasks["compile_experiment"].output  # noqa: N816
plot_simulation(
    new_compiled_T1,
    signal_names_to_show=["drive", "measure"],
    length=10e-6,
)

### Temporary settings

The qubit parameters are used to control the settings of pulses and instruments during the experiment. We can run the T1 experiment with different settings by passing it a copy of the qubits with modified parameters:

In [ ]:
# Make a copy of the qubits
temp_qubits = my_platform.qpu.copy_qubits()
# Change the length of the drive pulses
temp_qubits[0].parameters.ge_drive_length = 1000e-9
temp_qubits[1].parameters.ge_drive_length = 1000e-9

my_new_workflow = T1.experiment_workflow(
    session=session,
    qpu=my_platform.qpu,
    qubits=[temp_qubits[0], temp_qubits[1]],  # pass temporary qubits
    delays=[[0, 100e-9, 1000e-9], [0, 100e-9, 1000e-9]],
    options=my_new_opts,
)

my_new_results = my_new_workflow.run()
new_compiled_T1 = my_new_results.tasks["compile_experiment"].output  # noqa: N816
plot_simulation(
    new_compiled_T1,
    signal_names_to_show=["drive", "measure"],
    length=12e-6,
)

Great! You've now run your T1 experiment. Check out these other experiments to keep characterizing your qubits:

In [ ]:
# TODO: Add experiment links
# TODO: Add Analysis